In [6]:
!pip install ruamel.yaml

Defaulting to user installation because normal site-packages is not writeable


In [7]:
%env CUDA_VISIBLE_DEVICES=0

import torch
import glob
import os
#torch.load("")

def go(layers):
    for f in glob.glob("/mnt/ssd-1/igor/gpt-neox/models/dense_large.logit_lens.{}/*/*.pt".format(layers)):
        ch = torch.load(f, map_location="cpu")
        if 'args' in ch:
            del ch['optimizer']
            ch['args']['num_layers'] = layers
            os.rename(f, f + ".args_orig")
            torch.save(ch, f)
        del ch

env: CUDA_VISIBLE_DEVICES=0


In [ ]:
for i in range(25):
    go(i)

In [ ]:
for layers in range(25):
    for f in glob.glob("/mnt/ssd-1/igor/gpt-neox/models/dense_medium.{}/*/*.pt".format(layers)):
        ch = torch.load(f)
ch

In [14]:
import ruamel.yaml
import sys

In [28]:
import sys
from ruamel.yaml import YAML

inp = """\
{
   # parallelism settings ( you will want to change these based on your cluster setup, ideally scheduling pipeline stages
   # across the node boundaries )
   "pipe-parallel-size": 1,
}
"""

yaml = YAML()
code = yaml.load(inp)
yaml.dump(code, sys.stdout)

{pipe-parallel-size: 1}


In [33]:
import yaml
file_name = '/mnt/ssd-1/igor/gpt-neox/models/dense_medium/configs/config.yml'
with open(file_name) as conf_file:
    conf = yaml.load(conf_file, Loader=yaml.FullLoader)

print(conf['num-layers'])
conf['num-layers'] = 17
conf['load'] = "mypath"
conf['save'] = "mypath"

with open('/tmp/test.yml', 'w') as f:
    yaml.dump(conf, f)

24


In [13]:
# Rename wandb runs

import wandb
import json
import os

api = wandb.Api()
for run in api.runs("igoro/neox"):
    if run.name != "distill-0-0":
        print("Skipping", run.name)

    j = json.loads(run.json_config)
    if "load" in j and "value" in j["load"]:
        new_name = os.path.basename(j["load"]["value"])
    else:
        print("WARNING: Skipping", run)
        continue
    
    #run.name = new_name
    #run.update()


Skipping dense_small.out_linear_all.6
Skipping dense_small.dense_4h_to_h.3
Skipping dense_small.dense_4h_to_h.3
Skipping dense_small.dense_4h_to_h.3
Skipping dense_small.dense_4h_to_h.12
Skipping dense_small.dense_4h_to_h.11
Skipping dense_small.dense_4h_to_h.10
Skipping dense_small.dense_4h_to_h.9
Skipping dense_small.dense_4h_to_h.8
Skipping dense_small.dense_4h_to_h.7
Skipping dense_small.dense_4h_to_h.6
Skipping dense_small.dense_4h_to_h.5
Skipping dense_small.dense_4h_to_h.4
Skipping dense_small.dense_4h_to_h.3
Skipping dense_small.dense_4h_to_h.2
Skipping dense_small.dense_4h_to_h.1
Skipping dense_small.dense_4h_to_h.0
Skipping dense_small.dense_4h_to_h.1
Skipping dense_small.dense_4h_to_h.0
Skipping dense_small.dense_4h_to_h.0
Skipping dense_small.final_norm.12
Skipping dense_small.final_norm.11
Skipping dense_small.final_norm.10
Skipping dense_small.final_norm.9
Skipping dense_small.final_norm.8
Skipping dense_small.final_norm.7
Skipping dense_small.final_norm.6
Skipping dense_